In [1]:
#importing necessary libraries
from transformers import AutoProcessor, FlavaModel, FlavaImageModel, AutoImageProcessor, FlavaFeatureExtractor
import numpy as np
from torchmultimodal.models.flava.model import flava_model_for_classification
from transformers import FlavaProcessor, FlavaForPreTraining
import torchvision
import torch
import torch.nn as nn
import wandb



/home/IAIS/jraghu/.local/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
#Checking for GPU and settingt he device as GPU if avaialble
if torch.cuda.is_available():
    print("CUDA is available on this system.")
    device = torch.device("cuda")
else:
    print("CUDA is not available on this system.")
    device = torch.device("cpu")



CUDA is available on this system.


In [3]:
#Create a login artifact into wnadb logger to start logging of metrics
wandb.login()

Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.


wandb: Currently logged in as: jatinkarthik89 (continual_learning_flava). Use `wandb login --relogin` to force relogin


True

In [4]:
from tqdm import tqdm
from avalanche.benchmarks.generators import nc_benchmark, ni_benchmark
from avalanche.benchmarks.datasets import MNIST, CUB200

from torchvision.transforms import Compose, ToTensor, Normalize, RandomCrop
import torchvision.transforms as transforms
from torch.utils.data import random_split


resize_transform = transforms.Resize((224, 224))
grayscale_transform = transforms.Grayscale(num_output_channels=3)
train_transform = Compose([
    resize_transform,
    grayscale_transform,
    ToTensor()
    
])

test_transform = Compose([
    resize_transform,
    grayscale_transform,
    ToTensor()

])



cub_train = CUB200(
    './data/cub200', train=True, download=True, transform=train_transform
)

cub_test = CUB200(
    './data/cub200', train=False, download=True,  transform=test_transform
)

Files already downloaded and verified
Files already downloaded and verified


In [5]:
#Classsic class incremental creation from MNIST dataset
scenario_CIL = nc_benchmark(
    cub_train, cub_test, n_experiences=5, shuffle=True, seed=1234,
    task_labels=True
)


#Classsic Domain incremental creation from MNIST dataset
scenario_DIL = ni_benchmark(
    cub_train, cub_test, n_experiences=3, shuffle=False, seed=42,
    balance_experiences=True 
)

Frozen FLAVA model with a simple classification head

In [6]:
class flava_custom_with_sequence_length(torch.nn.Module):
    def __init__(self, num_classes, device='cuda'):
        super().__init__()
        self.image_processor = AutoImageProcessor.from_pretrained("facebook/flava-full")
        self.model_custom = FlavaImageModel.from_pretrained("facebook/flava-full").eval().to("cuda")

        self.mlp = nn.Sequential(
                nn.Linear(768*197, 1000),
                nn.ReLU(),
                nn.Linear(1000, 500),
                nn.ReLU(),
                nn.Linear(500, 200),
                nn.Sigmoid()
        )
        self.device = device

    def forward(self, x):
        if not isinstance(x, list):
            x = list(x.cuda())
         
        with torch.no_grad():
            inputs = self.image_processor(images=x, return_tensors="pt").to("cuda")
            outputs = self.model_custom(**inputs)
            
        features = outputs.last_hidden_state.view(outputs.last_hidden_state.size(0), -1)  # Flatten spatial dimensions
        logits = self.mlp(features)    
        
        #print("logits", logits.shape)
        return logits

In [7]:
Cl_model_with_seq = flava_custom_with_sequence_length(num_classes=200)

Could not find image processor class in the image processor config or the model config. Loading based on pattern matching with the model's feature extractor configuration.


In [8]:
from torch.optim import SGD
from torch.nn import CrossEntropyLoss
from avalanche.benchmarks.classic import SplitMNIST
from avalanche.evaluation.metrics import forgetting_metrics, accuracy_metrics, \
    loss_metrics, timing_metrics, cpu_usage_metrics, confusion_matrix_metrics, disk_usage_metrics, amca_metrics
from avalanche.models import SimpleMLP
from avalanche.logging import InteractiveLogger, TextLogger, TensorboardLogger, WandBLogger
from avalanche.evaluation.metrics.checkpoint import WeightCheckpoint
from avalanche.training.plugins import EvaluationPlugin, GEMPlugin, GDumbPlugin
from avalanche.training.supervised import Naive, ER_ACE, der, MER
from avalanche.training.templates import SupervisedTemplate
from transformers import AutoImageProcessor, ViTModel


# DEFINE THE EVALUATION PLUGIN and LOGGERS
# The evaluation plugin manages the metrics computation.
# It takes as argument a list of metrics, collectes their results and returns
# them to the strategy it is attached to.

# log to Tensorboard
tb_logger = TensorboardLogger()

# log to text file
text_logger = TextLogger(open('log.txt', 'a'))

# print to stdout
interactive_logger = InteractiveLogger()

#Chnage these wandb parameters for every separate run
wandb_logger = WandBLogger(
        project_name="FLAVA frozen model with CUB200 Naive CL",
        run_name="Second run_30 epochs",
        log_artifacts=True,
        path=None,
        
    )

eval_plugin = EvaluationPlugin(
    accuracy_metrics(minibatch=True, epoch=True, experience=True, stream=True),
    amca_metrics(),
    loss_metrics(minibatch=True, epoch=True, experience=True, stream=True),
    timing_metrics(epoch=True, epoch_running=True),
    forgetting_metrics(experience=True, stream=True),
    cpu_usage_metrics(experience=True),
    confusion_matrix_metrics(num_classes=scenario_DIL.n_classes, save_image=False,
                             stream=True),
    disk_usage_metrics(minibatch=True, epoch=True, experience=True, stream=True),
    loggers=[interactive_logger, text_logger, tb_logger, wandb_logger]
)


# CREATE THE STRATEGY INSTANCE (Naive)
cl_strategy_DIL = Naive(
    Cl_model_with_seq, SGD(Cl_model_with_seq.parameters(), lr=0.09, momentum=0.9),
    CrossEntropyLoss(), train_mb_size=100, train_epochs=30, eval_mb_size=25, 
    evaluator=eval_plugin, device=device)

# TRAINING LOOP
print('Starting experiment...')
results = []
for experience in scenario_DIL.train_stream:
    print("Start of experience: ", experience.current_experience)
    print("Current Classes: ", experience.classes_in_this_experience)

    # train returns a dictionary which contains all the metric values
    res = cl_strategy_DIL.train(experience)
    print('Training completed')

    print('Computing accuracy on the whole test set')
    # test also returns a dictionary which contains all the metric values
    results.append(cl_strategy_DIL.eval(scenario_DIL.test_stream))

Starting experiment...
Start of experience:  0
Current Classes:  [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99, 100, 101, 102, 103, 104, 105, 106, 107, 108, 109, 110, 111, 112, 113, 114, 115, 116, 117, 118, 119, 120, 121, 122, 123, 124, 125, 126, 127, 128, 129, 130, 131, 132, 133, 134, 135, 136, 137, 138, 139, 140, 141, 142, 143, 144, 145, 146, 147, 148, 149, 150, 151, 152, 153, 154, 155, 156, 157, 158, 159, 160, 161, 162, 163, 164, 165, 166, 167, 168, 169, 170, 171, 172, 173, 174, 175, 176, 177, 178, 179, 180, 181, 182, 183, 184, 185, 186, 187, 188, 189, 190, 191, 192, 193, 194, 195, 196, 197, 198, 199]
-- >> Start of training phase << --
100%|███

Unfrozen FLAVA model with simple Classification head on CUB200 dataset

In [8]:
class flava_custom_with_sequence_length_unfreeze(torch.nn.Module):
    def __init__(self, num_classes, device='cuda'):
        super().__init__()
        self.image_processor = AutoImageProcessor.from_pretrained("facebook/flava-full")
        self.model_custom = FlavaImageModel.from_pretrained("facebook/flava-full").eval().to("cuda")

        self.mlp = nn.Sequential(
                nn.Linear(768*197, 1000),
                nn.ReLU(),
                nn.Linear(1000, 500),
                nn.ReLU(),
                nn.Linear(500, 200),
                nn.Sigmoid()
        )
        self.device = device

    def forward(self, x):
        if not isinstance(x, list):
            x = list(x.cuda())
         
        #with torch.no_grad():
        inputs = self.image_processor(images=x, return_tensors="pt").to("cuda")
        outputs = self.model_custom(**inputs)
            
        features = outputs.last_hidden_state.view(outputs.last_hidden_state.size(0), -1)  # Flatten spatial dimensions
        logits = self.mlp(features)    
        
        #print("logits", logits.shape)
        return logits

In [7]:
unfreeze_CUB = flava_custom_with_sequence_length_unfreeze(num_classes=200)

Could not find image processor class in the image processor config or the model config. Loading based on pattern matching with the model's feature extractor configuration.


In [8]:
from torch.optim import SGD
from torch.nn import CrossEntropyLoss
from avalanche.benchmarks.classic import SplitMNIST
from avalanche.evaluation.metrics import forgetting_metrics, accuracy_metrics, \
    loss_metrics, timing_metrics, cpu_usage_metrics, confusion_matrix_metrics, disk_usage_metrics, amca_metrics
from avalanche.models import SimpleMLP
from avalanche.logging import InteractiveLogger, TextLogger, TensorboardLogger, WandBLogger
from avalanche.evaluation.metrics.checkpoint import WeightCheckpoint
from avalanche.training.plugins import EvaluationPlugin, GEMPlugin, GDumbPlugin
from avalanche.training.supervised import Naive, ER_ACE, der, MER
from avalanche.training.templates import SupervisedTemplate
from transformers import AutoImageProcessor, ViTModel


# DEFINE THE EVALUATION PLUGIN and LOGGERS
# The evaluation plugin manages the metrics computation.
# It takes as argument a list of metrics, collectes their results and returns
# them to the strategy it is attached to.

# log to Tensorboard
tb_logger = TensorboardLogger()

# log to text file
text_logger = TextLogger(open('log.txt', 'a'))

# print to stdout
interactive_logger = InteractiveLogger()

#Chnage these wandb parameters for every separate run
wandb_logger = WandBLogger(
        project_name="FLAVA un-frozen model with CUB200 Naive CL",
        run_name="First run_30epochs",
        log_artifacts=True,
        path=None,
        
    )

eval_plugin = EvaluationPlugin(
    accuracy_metrics(minibatch=True, epoch=True, experience=True, stream=True),
    amca_metrics(),
    loss_metrics(minibatch=True, epoch=True, experience=True, stream=True),
    timing_metrics(epoch=True, epoch_running=True),
    forgetting_metrics(experience=True, stream=True),
    cpu_usage_metrics(experience=True),
    confusion_matrix_metrics(num_classes=scenario_DIL.n_classes, save_image=False,
                             stream=True),
    disk_usage_metrics(minibatch=True, epoch=True, experience=True, stream=True),
    loggers=[interactive_logger, text_logger, tb_logger, wandb_logger]
)


# CREATE THE STRATEGY INSTANCE (Naive)
cl_strategy_DIL = Naive(
    unfreeze_CUB, SGD(unfreeze_CUB.parameters(), lr=0.09, momentum=0.9),
    CrossEntropyLoss(), train_mb_size=100, train_epochs=30, eval_mb_size=25, 
    evaluator=eval_plugin, device=device)

# TRAINING LOOP
print('Starting experiment...')
results = []
for experience in scenario_DIL.train_stream:
    print("Start of experience: ", experience.current_experience)
    print("Current Classes: ", experience.classes_in_this_experience)

    # train returns a dictionary which contains all the metric values
    res = cl_strategy_DIL.train(experience)
    print('Training completed')

    print('Computing accuracy on the whole test set')
    # test also returns a dictionary which contains all the metric values
    results.append(cl_strategy_DIL.eval(scenario_DIL.test_stream))

Starting experiment...
Start of experience:  0
Current Classes:  [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99, 100, 101, 102, 103, 104, 105, 106, 107, 108, 109, 110, 111, 112, 113, 114, 115, 116, 117, 118, 119, 120, 121, 122, 123, 124, 125, 126, 127, 128, 129, 130, 131, 132, 133, 134, 135, 136, 137, 138, 139, 140, 141, 142, 143, 144, 145, 146, 147, 148, 149, 150, 151, 152, 153, 154, 155, 156, 157, 158, 159, 160, 161, 162, 163, 164, 165, 166, 167, 168, 169, 170, 171, 172, 173, 174, 175, 176, 177, 178, 179, 180, 181, 182, 183, 184, 185, 186, 187, 188, 189, 190, 191, 192, 193, 194, 195, 196, 197, 198, 199]
-- >> Start of training phase << --
100%|███

Unfrozen FLAVA on CUB200 using ER CL strategy

In [9]:
unfreeze_CUB_ER = flava_custom_with_sequence_length_unfreeze(num_classes=200)

Could not find image processor class in the image processor config or the model config. Loading based on pattern matching with the model's feature extractor configuration.


In [10]:
from torch.optim import SGD
from torch.nn import CrossEntropyLoss
from avalanche.benchmarks.classic import SplitMNIST
from avalanche.evaluation.metrics import forgetting_metrics, accuracy_metrics, \
    loss_metrics, timing_metrics, cpu_usage_metrics, confusion_matrix_metrics, disk_usage_metrics, amca_metrics
from avalanche.models import SimpleMLP
from avalanche.logging import InteractiveLogger, TextLogger, TensorboardLogger, WandBLogger
from avalanche.evaluation.metrics.checkpoint import WeightCheckpoint
from avalanche.training.plugins import EvaluationPlugin, GEMPlugin, GDumbPlugin
from avalanche.training.supervised import Naive, ER_ACE, der, MER
from avalanche.training.templates import SupervisedTemplate
from transformers import AutoImageProcessor, ViTModel
from avalanche.training.plugins import ReplayPlugin


# DEFINE THE EVALUATION PLUGIN and LOGGERS
# The evaluation plugin manages the metrics computation.
# It takes as argument a list of metrics, collectes their results and returns
# them to the strategy it is attached to.

# log to Tensorboard
tb_logger = TensorboardLogger()

# log to text file
text_logger = TextLogger(open('log.txt', 'a'))

# print to stdout
interactive_logger = InteractiveLogger()

#Chnage these wandb parameters for every separate run
wandb_logger = WandBLogger(
        project_name="FLAVA un-frozen model with CUB200 Experience Replay CL",
        run_name="First run_30epochs",
        log_artifacts=True,
        path=None,
        
    )

eval_plugin = EvaluationPlugin(
    accuracy_metrics(minibatch=True, epoch=True, experience=True, stream=True),
    amca_metrics(),
    loss_metrics(minibatch=True, epoch=True, experience=True, stream=True),
    timing_metrics(epoch=True, epoch_running=True),
    forgetting_metrics(experience=True, stream=True),
    cpu_usage_metrics(experience=True),
    confusion_matrix_metrics(num_classes=scenario_DIL.n_classes, save_image=False,
                             stream=True),
    disk_usage_metrics(minibatch=True, epoch=True, experience=True, stream=True),
    loggers=[interactive_logger, text_logger, tb_logger, wandb_logger]
)


replay_plugin = ReplayPlugin(
        mem_size=50)


# CREATE THE STRATEGY INSTANCE (ER)
cl_strategy_DIL = Naive(
    unfreeze_CUB_ER, SGD(unfreeze_CUB_ER.parameters(), lr=0.09, momentum=0.9),
    CrossEntropyLoss(), train_mb_size=100, train_epochs=30, eval_mb_size=25, 
    evaluator=eval_plugin,plugins=[replay_plugin], device=device)

# TRAINING LOOP
print('Starting experiment...')
results = []
for experience in scenario_DIL.train_stream:
    print("Start of experience: ", experience.current_experience)
    print("Current Classes: ", experience.classes_in_this_experience)

    # train returns a dictionary which contains all the metric values
    res = cl_strategy_DIL.train(experience)
    print('Training completed')

    print('Computing accuracy on the whole test set')
    # test also returns a dictionary which contains all the metric values
    results.append(cl_strategy_DIL.eval(scenario_DIL.test_stream))

Starting experiment...
Start of experience:  0
Current Classes:  [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99, 100, 101, 102, 103, 104, 105, 106, 107, 108, 109, 110, 111, 112, 113, 114, 115, 116, 117, 118, 119, 120, 121, 122, 123, 124, 125, 126, 127, 128, 129, 130, 131, 132, 133, 134, 135, 136, 137, 138, 139, 140, 141, 142, 143, 144, 145, 146, 147, 148, 149, 150, 151, 152, 153, 154, 155, 156, 157, 158, 159, 160, 161, 162, 163, 164, 165, 166, 167, 168, 169, 170, 171, 172, 173, 174, 175, 176, 177, 178, 179, 180, 181, 182, 183, 184, 185, 186, 187, 188, 189, 190, 191, 192, 193, 194, 195, 196, 197, 198, 199]
-- >> Start of training phase << --
100%|███

Frozen FLAVA model on CUB200 dataset using ER CL Strategy

In [7]:
Cl_ER_frozen_cub = flava_custom_with_sequence_length(num_classes=200)

Could not find image processor class in the image processor config or the model config. Loading based on pattern matching with the model's feature extractor configuration.


In [8]:
from torch.optim import SGD
from torch.nn import CrossEntropyLoss
from avalanche.benchmarks.classic import SplitMNIST
from avalanche.evaluation.metrics import forgetting_metrics, accuracy_metrics, \
    loss_metrics, timing_metrics, cpu_usage_metrics, confusion_matrix_metrics, disk_usage_metrics, amca_metrics
from avalanche.models import SimpleMLP
from avalanche.logging import InteractiveLogger, TextLogger, TensorboardLogger, WandBLogger
from avalanche.evaluation.metrics.checkpoint import WeightCheckpoint
from avalanche.training.plugins import EvaluationPlugin, GEMPlugin, GDumbPlugin
from avalanche.training.supervised import Naive, ER_ACE, der, MER
from avalanche.training.templates import SupervisedTemplate
from transformers import AutoImageProcessor, ViTModel
from avalanche.training.plugins import ReplayPlugin


# DEFINE THE EVALUATION PLUGIN and LOGGERS
# The evaluation plugin manages the metrics computation.
# It takes as argument a list of metrics, collectes their results and returns
# them to the strategy it is attached to.

# log to Tensorboard
tb_logger = TensorboardLogger()

# log to text file
text_logger = TextLogger(open('log.txt', 'a'))

# print to stdout
interactive_logger = InteractiveLogger()

#Chnage these wandb parameters for every separate run
wandb_logger = WandBLogger(
        project_name="FLAVA frozen model with CUB200 Experience Replay CL",
        run_name="First run_30epochs",
        log_artifacts=True,
        path=None,
        
    )

eval_plugin = EvaluationPlugin(
    accuracy_metrics(minibatch=True, epoch=True, experience=True, stream=True),
    amca_metrics(),
    loss_metrics(minibatch=True, epoch=True, experience=True, stream=True),
    timing_metrics(epoch=True, epoch_running=True),
    forgetting_metrics(experience=True, stream=True),
    cpu_usage_metrics(experience=True),
    confusion_matrix_metrics(num_classes=scenario_DIL.n_classes, save_image=False,
                             stream=True),
    disk_usage_metrics(minibatch=True, epoch=True, experience=True, stream=True),
    loggers=[interactive_logger, text_logger, tb_logger, wandb_logger]
)


replay_plugin = ReplayPlugin(
        mem_size=50)


# CREATE THE STRATEGY INSTANCE (ER)
cl_strategy_DIL = Naive(
    Cl_ER_frozen_cub, SGD(Cl_ER_frozen_cub.parameters(), lr=0.09, momentum=0.9),
    CrossEntropyLoss(), train_mb_size=100, train_epochs=30, eval_mb_size=25, 
    evaluator=eval_plugin,plugins=[replay_plugin], device=device)

# TRAINING LOOP
print('Starting experiment...')
results = []
for experience in scenario_DIL.train_stream:
    print("Start of experience: ", experience.current_experience)
    print("Current Classes: ", experience.classes_in_this_experience)

    # train returns a dictionary which contains all the metric values
    res = cl_strategy_DIL.train(experience)
    print('Training completed')

    print('Computing accuracy on the whole test set')
    # test also returns a dictionary which contains all the metric values
    results.append(cl_strategy_DIL.eval(scenario_DIL.test_stream))

Starting experiment...
Start of experience:  0
Current Classes:  [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99, 100, 101, 102, 103, 104, 105, 106, 107, 108, 109, 110, 111, 112, 113, 114, 115, 116, 117, 118, 119, 120, 121, 122, 123, 124, 125, 126, 127, 128, 129, 130, 131, 132, 133, 134, 135, 136, 137, 138, 139, 140, 141, 142, 143, 144, 145, 146, 147, 148, 149, 150, 151, 152, 153, 154, 155, 156, 157, 158, 159, 160, 161, 162, 163, 164, 165, 166, 167, 168, 169, 170, 171, 172, 173, 174, 175, 176, 177, 178, 179, 180, 181, 182, 183, 184, 185, 186, 187, 188, 189, 190, 191, 192, 193, 194, 195, 196, 197, 198, 199]
-- >> Start of training phase << --
100%|███